## 一、`nn.Module` 是什么？

`nn.Module` 是一个“带参数的可调用对象”的基类，它可以：
* 管理模型里的**参数**（权重、偏置）
* 管理模型里的**子模块**（层、子网络）
* 统一提供一些方法（`parameters()`、`to()`、`train()`、`eval()` 等）

我们的网络基本都长这样：

```python
class MyNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 在这里定义“结构”（子模块 / 层）
    
    def forward(self, x):
        # 在这里定义“数据是如何流过这些结构的”
        return x
```

* `__init__` 里定义**需要学习的层 / 子模块**（`nn.Linear`, `nn.Conv2d` 等）
* `forward` 里定义**前向计算流程**
* 不需要手写 `backward`，PyTorch 自动利用 autograd 通过 `forward` 的计算图求梯度。


## 二. 一个使用模板

```py
import torch
from torch import nn

class MLP(nn.Module):
    def __init__(self):
        super().__init__()  # 初始化基类
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)               # [batch, 1, 28, 28] -> [batch, 784]
        logits = self.linear_relu_stack(x)
        return logits                     # [batch, 10]

model = MLP()
```
注意：
1. `class MLP(nn.Module)`：继承基类
2. `super().__init__()`：调用父类构造函数（不写会出各种怪问题）
3. 在 __init__ 里把层“挂”在 `self.xxx` 上：
    - 只要是赋给 `self. `的 `nn.Module` 或 `nn.Parameter`，PyTorch 会自动登记为参数 / 子模块
4. 在 `forward` 里使用这些层完成前向计算

## 三. 子模块和`nn.Sequential`

### 3.1 如果不使用`nn.Sequential`的话，也可以这样写
```py
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x
```

这里每个 `self.fc1`, `self.fc2`, `self.fc3` 都是子模块，`SimpleNet` 是一个“大模块”，它管理所有子模块的参数。

### 3.2 利用`nn.Sequential`(类似于搭积木)

`nn.Sequential` 本质就是一个“按顺序执行的一串模块”：

```py
self.linear_relu_stack = nn.Sequential(
    nn.Linear(28 * 28, 512),
    nn.ReLU(),
    nn.Linear(512, 512),
    nn.ReLU(),
    nn.Linear(512, 10)
)
```

它的`forward`相当于

```py
def forward(self, x):
    for layer in self.layers:
        x = layer(x)
    return x
```

## 四、`nn.Module` 和训练循环的关系

### 4.1 `parameters()`：交给优化器的就是它

当我们写：

```python
model = MLP().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
```

这里的 `model.parameters()` 就是 `nn.Module` 提供的一个方法：

* 会自动遍历所有**子模块**，收集里面的 `nn.Parameter`（可学习的权重）
* 返回一个可迭代对象，优化器就能更新这些参数

所以你只要**正确地在 `__init__` 里挂层到 `self.xxx` 上**，剩下交给 `model.parameters()` 就行。

---

### 4.2 `to()` / `cuda()`：模型整体搬设备

```python
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MLP().to(device)
```

* `model.to(device)` 会把所有参数和缓冲（如 `running_mean` 等）都搬到对应设备
* 之后我们只要保证输入 `x` 也 `.to(device)`，就能在 GPU 上跑前向 + 反向

---

### 4.3 `train()` / `eval()`：切换训练 / 测试模式

```python
model.train()   # 训练模式
# ...
model.eval()    # 测试/推理模式
```

这两个方法是 `nn.Module` 提供的，它们会：

* 设置 `self.training = True/False`
* 递归调用所有子模块的 `train()` / `eval()`

为什么需要这个？

* 像 `nn.Dropout`、`nn.BatchNorm` 这种层在 train/eval 下行为不同：

  * Dropout：训练时随机置零部分神经元，测试时关闭随机（使用缩放后的期望）
  * BatchNorm：训练时用 batch 的统计量，测试时用滑动平均统计量

如果我们不写 `model.train()` / `model.eval()`：

* 在测试时 Dropout 仍然随机，会导致预测不稳定
* BatchNorm 不会正确使用训练好的统计量